### 모델링 기법 : 발생건수 15~20년의 어린이 보호구역 내 어린이사고 데이터를 시계열로 분류해 클러스터링 기법을 활용하여 사고위험도를 높음/보통/낮음 등으로 군집화(+시각화) -> 위의 데이터를 머신러닝을 통해 1시간 뒤의 어린이 보호구역 사고 위험도 예측
### 이때 위험도 즉 사고지수는 보호구역 내 사고 수  * 사고수준(중상,경상) * 차종(이륜, 화물) 같은 방식. 

### 랜덤 포레스트 변수 중요도 분석을 통해 어린이 보호구역 내 어린이사고에 큰 영향을 미치는 요인 분석(+시각화)

## -> 사고위험도가 높은 보호구역 중 변수 중요도의 상위 부분을 차지하는 요인 개선사항 제안 -> 머신러닝을 통해 우리가 제시한 개선사항 개선 후 사고위험도 예측


In [2]:
# 기본 패키지
import numpy as np 
import pandas as pd

In [3]:
# csv 파일 불러오기
speed_sample03 = pd.read_csv('C:/Users/wnrrh/desktop/data/sample/속도샘플.csv',header=0,encoding='utf-8')
df = pd.read_csv('C:/Users/wnrrh/desktop/data/real/어린이보호구역현황.csv',header=0,encoding='utf-8')
accident = pd.read_csv('C:/Users/wnrrh/desktop/data/real/어린이보호구역내사고.csv',header=0,encoding='utf-8')
dfplus = pd.read_csv('C:/Users/wnrrh/desktop/data/real/광진도봉 어린이집 현황.csv',header=0,encoding='utf-8')

In [4]:
columns = ["일자","요일","도로명","링크아이디","시점명","종점명","방향","거리","차선수","기능유형구분",
          "도심/외곽구분", " 권역구분", "01시", "02시","03시","04시","05시","06시"
          ,"07시","08시","09시","10시","11시","12시","13시","14시","15시","16시",
          "17시","18시","19시","20시","21시","22시","23시","24시"]
speed_sample03.columns = columns


In [5]:
speed_sample03.drop(["링크아이디"],axis=1, inplace = True)
speed_sample03

,일자,요일,도로명,시점명,종점명,방향,거리,차선수,기능유형구분,도심/외곽구분,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
0,20210301,월,4.19로,아카데미하우스,국립4.19묘지,상행,1179,1,보조간선도로,외곽,...,38.29,34.70,56.64,46.43,35.58,39.23,33.28,48.66,38.08,45.81
1,20210301,월,4.19로,국립4.19묘지,국립4.19묘지입구,상행,352,2,보조간선도로,외곽,...,13.66,12.32,14.36,16.68,16.53,17.05,14.71,17.96,17.96,18.66
2,20210301,월,4.19로,국립4.19묘지입구,국립4.19묘지,하행,354,2,보조간선도로,외곽,...,20.70,19.20,23.24,20.96,21.93,22.10,22.63,18.78,21.25,26.87
3,20210301,월,남대문로,한국은행앞,숭례문,상행,533,3,보조간선도로,도심권,...,20.71,20.62,21.15,22.74,23.52,22.73,24.88,25.70,24.51,24.83
4,20210301,월,남대문로,숭례문,한국은행앞,하행,544,2,보조간선도로,도심권,...,16.23,21.35,21.55,24.52,25.21,24.38,26.53,25.51,24.83,22.45
5,20210301,월,가락로,방이동고분군,송파한양아파트,상행,416,1,기타도로,외곽,...,21.47,22.10,23.97,24.41,23.20,21.79,20.45,24.05,26.20,27.54
6,20210301,월,도당로,성원아파트,방학동새마을금고,상행,299,2,기타도로,외곽,...,14.97,13.45,15.23,16.62,16.89,11.03,16.56,21.25,19.42,15.07
7,20210301,월,도당로,방학동새마을금고,방학역,상행,720,2,기타도로,외곽,...,17.44,18.95,21.78,17.15,20.36,18.89,18.99,18.31,21.61,20.37
8,20210328,일,선릉로,한티역,도곡역,상행,636,3,기타도로,외곽,...,22.00,21.36,19.28,21.27,22.66,21.64,24.36,26.99,28.04,33.52
9,20210328,일,선릉로,도곡역,영동4교,상행,441,2,기타도로,외곽,...,30.26,29.36,28.80,27.31,28.37,29.58,30.23,31.46,32.23,35.62


In [6]:
columns = ["시설종류","대상시설명","소재지도로명주소","소재지지번주소","위도",
          "경도","관리기관명","관할경찰서","CCTV설치여부","CCTV설치대수",
           "보호구역도로폭","데이터기준일자","제공기관코드","제공기관명"]
df.columns = columns
df

,시설종류,대상시설명,소재지도로명주소,소재지지번주소,위도,경도,관리기관명,관할경찰서,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자,제공기관코드,제공기관명
0,어린이집,에반어린이집,서울특별시 강서구 강서로12길 23-7(화곡동),NaN,37.532112,126.847959,서울특별시 강서구청,강서경찰서,Y,1.0,6,2021-08-17,3150000,서울특별시 강서구
1,어린이집,구립까치산어린이집,서울특별시 강서구 강서로12길 29-10(화곡동),NaN,37.532383,126.848515,서울특별시 강서구청,강서경찰서,Y,1.0,6,2021-08-17,3150000,서울특별시 강서구
2,어린이집,배다리어린이집,서울특별시 강서구 곰달래로15길 25(화곡동),NaN,37.531278,126.839543,서울특별시 강서구청,강서경찰서,N,NaN,6,2021-08-17,3150000,서울특별시 강서구
3,어린이집,버클리어린이집,서울특별시 강서구 강서로47바길 21(발산1동),NaN,37.552473,126.830123,서울특별시 강서구청,강서경찰서,Y,1.0,6,2021-08-17,3150000,서울특별시 강서구
4,어린이집,구립새하늘어린이집,서울특별시 강서구 강서로17다길 15-14(화곡동),NaN,37.533307,126.841415,서울특별시 강서구청,강서경찰서,N,NaN,6,2021-08-17,3150000,서울특별시 강서구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1641,유치원,방배병설유치원,서울특별시 서초구 방배중앙로13길 26,서울특별시 서초구 방배동 861-1,37.486764,126.987137,서울시 서초구,방배경찰서,Y,1.0,1,2020-07-01,3210000,서울특별시 서초구
1642,유치원,새성민유치원,서울특별시 서초구 효령로 110,서울특별시 서초구 방배동 981-2,37.480657,126.996119,서울시 서초구,방배경찰서,Y,1.0,1,2020-07-01,3210000,서울특별시 서초구
1643,유치원,새순유치원,서울특별시 서초구 서초대로15길 57,서울특별시 서초구 방배동 831-12,37.489064,126.988580,서울시 서초구,방배경찰서,Y,1.0,1,2020-07-01,3210000,서울특별시 서초구
1644,유치원,서래병설유치원,서울특별시 서초구 방배로 260,서울특별시 서초구 방배본동 771,37.496405,126.990157,서울시 서초구,방배경찰서,Y,1.0,1,2020-07-01,3210000,서울특별시 서초구


In [7]:
df.drop(["소재지지번주소","관리기관명","관할경찰서","데이터기준일자","제공기관코드","CCTV설치대수","시설종류",
        "제공기관명"],axis=1, inplace=True)

In [8]:
data_list = df['소재지도로명주소'].str.split(' ')
df['자치구'] = data_list.str.get(1)
df

,대상시설명,소재지도로명주소,위도,경도,CCTV설치여부,보호구역도로폭,자치구
0,에반어린이집,서울특별시 강서구 강서로12길 23-7(화곡동),37.532112,126.847959,Y,6,강서구
1,구립까치산어린이집,서울특별시 강서구 강서로12길 29-10(화곡동),37.532383,126.848515,Y,6,강서구
2,배다리어린이집,서울특별시 강서구 곰달래로15길 25(화곡동),37.531278,126.839543,N,6,강서구
3,버클리어린이집,서울특별시 강서구 강서로47바길 21(발산1동),37.552473,126.830123,Y,6,강서구
4,구립새하늘어린이집,서울특별시 강서구 강서로17다길 15-14(화곡동),37.533307,126.841415,N,6,강서구
...,...,...,...,...,...,...,...
1641,방배병설유치원,서울특별시 서초구 방배중앙로13길 26,37.486764,126.987137,Y,1,서초구
1642,새성민유치원,서울특별시 서초구 효령로 110,37.480657,126.996119,Y,1,서초구
1643,새순유치원,서울특별시 서초구 서초대로15길 57,37.489064,126.988580,Y,1,서초구
1644,서래병설유치원,서울특별시 서초구 방배로 260,37.496405,126.990157,Y,1,서초구


In [9]:
df.drop(['소재지도로명주소'],axis=1, inplace = True)


In [10]:
df = df[['대상시설명','자치구','CCTV설치여부','보호구역도로폭','위도','경도']]
df

,대상시설명,자치구,CCTV설치여부,보호구역도로폭,위도,경도
0,에반어린이집,강서구,Y,6,37.532112,126.847959
1,구립까치산어린이집,강서구,Y,6,37.532383,126.848515
2,배다리어린이집,강서구,N,6,37.531278,126.839543
3,버클리어린이집,강서구,Y,6,37.552473,126.830123
4,구립새하늘어린이집,강서구,N,6,37.533307,126.841415
...,...,...,...,...,...,...
1641,방배병설유치원,서초구,Y,1,37.486764,126.987137
1642,새성민유치원,서초구,Y,1,37.480657,126.996119
1643,새순유치원,서초구,Y,1,37.489064,126.988580
1644,서래병설유치원,서초구,Y,1,37.496405,126.990157


In [11]:
columns = ["대상시설명","시설종류","소재지도로명주소","구역지정수","CCTV 수","시설물 도로표지판 수",
          "시설물 과속방지턱 수","18-20 사고발생건수","15-17 사고발생건수"]
dfplus.columns = columns

In [12]:
dfplus['소재지도로명주소'] = dfplus['소재지도로명주소'].astype('str')
dfplus['소재지도로명주소'].dtypes
data_list = dfplus['소재지도로명주소'].str.split(' ')
data_list.head()
dfplus['자치구'] = data_list.str.get(1)
dfplus

,대상시설명,시설종류,소재지도로명주소,구역지정수,CCTV 수,시설물 도로표지판 수,시설물 과속방지턱 수,18-20 사고발생건수,15-17 사고발생건수,자치구
0,보성유치원,유치원(일반),"서울특별시 광진구 아차산로 552 (광장동, 광장극동아파트)",1,0,0,0,0,1,광진구
1,광장유치원,유치원(일반),서울특별시 광진구 광장로7길 17 (광장동),2,0,0,4,0,0,광진구
2,서울광장초등학교,초등학교,서울특별시 광진구 광장로1길 27 (광장동),2,0,0,8,0,0,광진구
3,양진초등학교,초등학교,서울특별시 광진구 워커힐로 32 (광장동),4,0,0,12,0,3,광진구
4,양진병설유치원,유치원(병설),서울특별시 광진구 워커힐로 32 (광장동),0,0,0,0,0,3,광진구
...,...,...,...,...,...,...,...,...,...,...
324,한별유치원,유치원(일반),서울특별시 도봉구 덕릉로 63가길 43 (창동),2,0,0,2,0,0,도봉구
325,자연유치원,유치원(일반),서울특별시 도봉구 마들로 567 (창동),2,0,0,2,0,1,도봉구
326,청아유치원,유치원(일반),"서울특별시 도봉구 노해로69가길 11 (창동, 동아청솔아파트)",2,0,0,0,0,0,도봉구
327,삼풍유치원,유치원(일반),서울특별시 도봉구 노해로63가길 101 (창동),3,0,0,5,0,1,도봉구


In [13]:
dfplus.drop(["18-20 사고발생건수","15-17 사고발생건수"],axis=1, inplace=True)
dfplus

,대상시설명,시설종류,소재지도로명주소,구역지정수,CCTV 수,시설물 도로표지판 수,시설물 과속방지턱 수,자치구
0,보성유치원,유치원(일반),"서울특별시 광진구 아차산로 552 (광장동, 광장극동아파트)",1,0,0,0,광진구
1,광장유치원,유치원(일반),서울특별시 광진구 광장로7길 17 (광장동),2,0,0,4,광진구
2,서울광장초등학교,초등학교,서울특별시 광진구 광장로1길 27 (광장동),2,0,0,8,광진구
3,양진초등학교,초등학교,서울특별시 광진구 워커힐로 32 (광장동),4,0,0,12,광진구
4,양진병설유치원,유치원(병설),서울특별시 광진구 워커힐로 32 (광장동),0,0,0,0,광진구
...,...,...,...,...,...,...,...,...
324,한별유치원,유치원(일반),서울특별시 도봉구 덕릉로 63가길 43 (창동),2,0,0,2,도봉구
325,자연유치원,유치원(일반),서울특별시 도봉구 마들로 567 (창동),2,0,0,2,도봉구
326,청아유치원,유치원(일반),"서울특별시 도봉구 노해로69가길 11 (창동, 동아청솔아파트)",2,0,0,0,도봉구
327,삼풍유치원,유치원(일반),서울특별시 도봉구 노해로63가길 101 (창동),3,0,0,5,도봉구


In [14]:
dataframe = pd.merge(df,dfplus, on=['대상시설명','자치구'])
dataframe = dataframe[['대상시설명','소재지도로명주소','자치구','시설종류','구역지정수',
                       'CCTV설치여부','CCTV 수','보호구역도로폭','시설물 도로표지판 수'
                       ,'시설물 과속방지턱 수','위도','경도']]
dataframe

,대상시설명,소재지도로명주소,자치구,시설종류,구역지정수,CCTV설치여부,CCTV 수,보호구역도로폭,시설물 도로표지판 수,시설물 과속방지턱 수,위도,경도
0,경일어린이집,서울특별시 도봉구 도당로13길 34-5 (방학동),도봉구,어린이집,1,Y,0,6,0,2,37.666418,127.034903
1,연촌초등학교,서울특별시 노원구 공릉로58길 36 (하계동),노원구,초등학교,3,Y,4,7~15,10,4,37.634416,127.072512
2,신창어린이집,서울특별시 도봉구 덕릉로62길 26 (창동),도봉구,어린이집,1,Y,0,8,0,0,37.640093,127.041305
3,창4동어린이집,서울특별시 도봉구 마들로 339-13 (창동),도봉구,어린이집,1,Y,0,8,0,4,37.642164,127.053289
4,꿈동산몬테소리어린이집,서울특별시 광진구 면목로4길 3 (군자동),광진구,어린이집,2,Y,0,NaN,0,0,37.555820,127.076065
...,...,...,...,...,...,...,...,...,...,...,...,...
323,코레일오류동어린이집,서울특별시 구로구 경인로20가길 24 (오류동),구로구,어린이집,2,Y,0,NaN,0,0,37.495483,126.845330
324,항동어린이집,서울특별시 구로구 연동로 284 (항동),구로구,어린이집,1,Y,0,NaN,0,4,37.485379,126.823438
325,개봉초등학교 병설유치원,서울특별시 구로구 개봉로16길 30-11 (개봉동),구로구,유치원(병설),0,Y,0,NaN,0,0,37.489523,126.857975
326,서울고척초등학교 병설유치원,서울특별시 구로구 경서로 31 (고척동),구로구,유치원(병설),0,Y,0,NaN,0,0,37.500947,126.856994


In [15]:
dataframe.to_excel('C:/Users/wnrrh/desktop/data/sample/광진도봉.xlsx',encoding='cp949')

In [51]:
columns = ["사고번호","사고일시","요일","시군구","사고내용","사망자수","중상자수",
          "경상자수","부상신고자수","사고유형","법규위반","노면상태","기상상태",
          "도로형태","가해운전자 차종","가해운전자 성별","가해운전자 연령","가해운전자 상해정도",
          "피해운전자 차종","피해운전자 성별","피해운전자 연령","피해운전자 상해정도"]
accident.columns = columns
accident

,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,...,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도
0,2.015010e+15,2015년 1월 6일 05시,화요일,서울특별시 용산구 후암동,사망사고,1,0,0,0,차대사람 - 기타,...,맑음,교차로 - 교차로안,승용,남,22세,상해없음,보행자,남,75세,사망
1,2.015010e+15,2015년 1월 7일 15시,수요일,서울특별시 종로구 계동,중상사고,0,1,0,0,차대사람 - 차도통행중,...,맑음,단일로 - 기타,승용,남,70세,상해없음,보행자,여,16세,중상
2,2.015010e+15,2015년 1월 14일 09시,수요일,서울특별시 관악구 신림동,경상사고,0,0,1,0,차대사람 - 기타,...,맑음,교차로 - 교차로안,승용,남,70세,상해없음,보행자,남,10세,경상
3,2.015010e+15,2015년 1월 14일 13시,수요일,서울특별시 중랑구 면목동,경상사고,0,0,1,0,차대사람 - 기타,...,맑음,단일로 - 기타,승용,남,29세,상해없음,보행자,남,10세,경상
4,2.015010e+15,2015년 1월 26일 08시,월요일,서울특별시 양천구 신정동,경상사고,0,0,1,0,차대사람 - 기타,...,흐림,단일로 - 기타,승용,기타불명,기타불명,기타불명,보행자,여,20세,경상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1391,2.020120e+15,2020년 12월 20일 23시,일요일,서울특별시 관악구 봉천동,경상사고,0,0,1,0,차대차 - 기타,...,맑음,단일로 - 기타,이륜,남,41세,경상,승용,남,51세,상해없음
1392,2.020120e+15,2020년 12월 21일 21시,월요일,서울특별시 성북구 석관동,경상사고,0,0,1,0,차대사람 - 횡단중,...,맑음,단일로 - 기타,이륜,남,37세,상해없음,보행자,남,33세,경상
1393,2.020120e+15,2020년 12월 28일 13시,월요일,서울특별시 동작구 대방동,경상사고,0,0,1,0,차대차 - 추돌,...,맑음,단일로 - 기타,승합,남,29세,상해없음,승용,남,71세,상해없음
1394,2.020120e+15,2020년 12월 28일 13시,월요일,서울특별시 은평구 갈현동,경상사고,0,0,1,0,차대차 - 측면충돌,...,맑음,교차로 - 교차로안,승용,여,29세,상해없음,자전거,남,88세,경상
